In [ ]:
using MolecularEvolution

In [ ]:
#functions used later in forward and backward

#building eta  - the vector of equilibrium frequencies according to group
function eta_calc(A::Vector{Vector{Int64}}, Pi::Vector{Float64})
    piNormed = Pi ./ sum(Pi)
    eta = copy(piNormed)
    for x in A
        eta[x] .*= 1/sum(eta[x])
    end
    return(eta)
end

#sigma and xi
function sigma_calc(A::Vector{Vector{Int64}}, V::Vector{Float64})
    sigma_vec=Array{Float64,1}(undef, length(A))
    count=1
    for x in A
        s=0.0
        for i in x
            s+=V[i]
        end
        sigma_vec[count]=s
        count+=1
    end
    return(sigma_vec)
end


function xi_calc(A::Vector{Vector{Int64}}, V::Array{Float64,1}, eta_vec::Array{Float64,1})
    xi_vec=Array{Float64,1}(undef, length(A))
    count=1
    for x in A
        s=0.0
        for i in x
            s+=V[i]*eta_vec[i]
        end
        xi_vec[count]=s
        count+=1
    end    
    return(xi_vec)
end



In [ ]:
states = 100
sites = 1 #For the case of geographic data, there is only one site for each sequence, however the model is generalizable to cases with more then one site
num_groups = 10 

#For now substitution rate parameters and equilibrium frequencies are randomly assigned
u1_val = 0.5
u2_val = 0.7
Pi_vals = rand(states) .+ 0.5
Pi_vals .= Pi_vals./sum(Pi_vals)


#The variable 'structure' stores the hierarchical structure within the data and passes it into the model
alloc = rand(1:num_groups,states)
structure = [findall(alloc .== g) for g in 1:num_groups]
structure = structure[length.(structure) .> 0];

eta=eta_calc(structure, Pi_vals);

In [ ]:
#Building the model itself
mutable struct HQtPiModel <: DiscreteStateModel
    u1::Float64 #local transition rate
    u2::Float64 #global transition rate
    Pi::Array{Float64, 1} #equilibrium frequencies
    Structure::Vector{Vector{Any}} #
    Eta::Array{Float64, 1} #Group normalisations of Pi
end
model=HQtPiModel(u1_val,u2_val,Pi_vals, structure, eta)


#defining the forward function f(t)
function MolecularEvolution.forward!(dest::CustomDiscretePartition,
        source::CustomDiscretePartition,
        model::HQtPiModel,
        node::FelNode)
    p1=exp(-model.u1*node.branchlength); p2=exp(-model.u2*node.branchlength)
    q1=1-p1; q2=1-p2 
    for j in 1:dest.sites
        dest.state[:,j] .= p1*p2 .* source.state[:,j] .+ model.Pi .* (q2*sum(source.state[:,j]))
        sigma=sigma_calc(model.Structure, source.state[:,j])
        for x in 1:length(model.Structure)
            for i in model.Structure[Int(x)]
                dest.state[i,j] += q1*p2.* sigma[Int(x)] .* model.Eta[i]
            end
        end
    end
    
    dest.scaling .= source.scaling
end

#defining the backward function b(t)
function MolecularEvolution.backward!(dest::CustomDiscretePartition,
        source::CustomDiscretePartition,
        model::HQtPiModel,
        node::FelNode)
    p1=exp(-model.u1*node.branchlength); p2=exp(-model.u2*node.branchlength)
    q1=1-p1; q2=1-p2 
    for j in 1:dest.sites
        dest.state[:,j] .= q2 * sum(source.state[:,j] .* model.Pi) .+ p1*p2 .* source.state[:,j]
        xi=xi_calc(model.Structure, source.state[:,j], model. Eta)
        for x in 1:length(model.Structure)
            for i in model.Structure[Int(x)]
                dest.state[i,j] += q1*p2 * xi[Int(x)] 
            end
        end
    end
    dest.scaling .= source.scaling
end


In [ ]:
#constructing a simulated tree on which to run the model
tre = sim_tree(500,500.0,0.25, mutation_rate = 0.01)

In [ ]:
test_partition = CustomDiscretePartition(states,sites)
internal_message_init!(tre,test_partition)
tre.parent_message[1].state .= Pi_vals
sample_down!(tre, model)

In [ ]:
#testing runtimes for three commands
log_likelihood!(tre,model)
@time a = log_likelihood!(tre,model)
felsenstein_down!(tre,model)
@time felsenstein_down!(tre,model)
felsenstein!(tre,model)
@time felsenstein!(tre,model)